# Synopsis

Create an LDA topic model from scratch using collapsed Gibbs Sampling.

# Configuration

In [143]:
base_path = '/Users/rca2t/COURSES/DSI/DS5559/UVA_DSI_REPO'
local_lib = base_path + '/lib'
src_dir = base_path + '/play/lda/corpora'
corpus_db = "20news.db"

In [144]:
n_docs = 50
n_topics = 20
n_iters = 100
alpha = 1
beta = .001

# Libraries

In [145]:
import pandas as pd
import numpy as np
import sqlite3
import re
import random
import sys; sys.path.append(local_lib)
import textman.textman as tx

# Process

## Get corpus

In [146]:
sql = "SELECT * FROM doc ORDER BY RANDOM() LIMIT ?"
with sqlite3.connect(src_dir + "/" + corpus_db) as db:
    docs = pd.read_sql(sql, db, index_col='doc_id', params=(n_docs,))

In [147]:
labels = set([topic for group in docs.doc_label.unique() for topic in group.split('.')[1:] if topic not in ['x']])

In [148]:
' '.join(labels)

'politics windows mac ms-windows space os med autos hardware mideast misc sport baseball ibm christian pc crypt sys electronics motorcycles religion hockey'

In [149]:
docs.doc_label.value_counts()

comp.sys.ibm.pc.hardware    8
sci.electronics             5
rec.sport.hockey            5
comp.sys.mac.hardware       5
soc.religion.christian      4
rec.motorcycles             4
rec.autos                   4
talk.politics.misc          3
comp.windows.x              2
talk.religion.misc          2
sci.crypt                   2
sci.med                     2
rec.sport.baseball          1
comp.os.ms-windows.misc     1
sci.space                   1
talk.politics.mideast       1
Name: doc_label, dtype: int64

In [150]:
# n_topics = len(labels)
# n_topics

## Convert corpus to tokens and vocab

In [151]:
tokens, vocab = tx.create_tokens_and_vocab(docs, src_col='doc_content')
tokens['token_num'] = tokens.groupby(['doc_id']).cumcount()
tokens = tokens.reset_index()[['doc_id','token_num','term_id']]
tokens = tokens[tokens.term_id.isin(vocab[vocab.go].index)]
tokens = tokens.set_index(['doc_id','token_num'])

In [152]:
tokens['term_str'] = tokens.term_id.map(vocab.term)

In [153]:
sw = """people one would know like men think two see well said years guy things want let get back new first make even take 
going right way could everything need says also look got made say used man every real anotherthree use help means good 
never took went tell day old ever told nobody show knew nothing five big fit still turn give found later began 
another sure since mit edu com gov however""".split()

In [154]:
tokens = tokens[~tokens.term_str.isin(sw)]

In [155]:
tokens.head()

term_id term_str
doc_id token_num                  
52801  0              386  article
       3             2924   writes
       4             1719    might
       5              258      add
       6             2940     year

## Reduce tokens to unique values

In [98]:
# tokens = tokens.groupby(['doc_id','term_id']).count().rename(columns={'term_str':'n'})

In [99]:
# tokens.head()

## Create topics table

In [170]:
topics = pd.DataFrame(index=range(n_topics))
topics.index.name = 'topic_id'

## Randomly assign topics to tokens

In [171]:
def init_topics(tokens):
    tokens['topic_id'] = np.random.randint(0, n_topics, len(tokens))
    return tokens

In [172]:
tokens = init_topics(tokens)

In [173]:
tokens.head()

term_id term_str  topic_id
doc_id token_num                            
52801  0              386  article         4
       3             2924   writes        19
       4             1719    might        16
       5              258      add        10
       6             2940     year        13

### Generate Count Matrices

In [174]:
def get_DT(tokens):
    return tokens.groupby(['doc_id','topic_id']).topic_id.count()\
        .unstack().fillna(0).astype('int')

In [175]:
def get_WT(tokens):
    return tokens.groupby(['topic_id','term_id']).term_id.count()\
        .unstack().fillna(0).astype('int').T

In [176]:
def init_theta(dt):
    m = np.ones(topics.shape[0])
    theta = dt.apply(lambda x: pd.Series(np.random.dirichlet(alpha * m)), 1)
    theta.columns.name = 'topic_id'
    return theta

## METHOD 1

See [this blog](http://brooksandrew.github.io/simpleblog/articles/latent-dirichlet-allocation-under-the-hood/).

### Define row function

In [179]:
def get_new_topic(row):
    
    d = row.name[0] # doc_id
    k = row.name[1] # token_num
    z = row.topic_id
    w = row.term_id
    
    DT.at[d,z] = DT.at[d,z] - 1
    WT.at[w,z] = WT.at[w,z] - 1
    
    DTP = DT.loc[d] / DT.loc[d].sum()
    WTP = WT.loc[w] / WT.sum() #WT.loc[w].sum()
        
    p_z = DTP * WTP
    z_weights = p_z / p_z.sum()

    z1 = topics.sample(weights=z_weights).index.values[0]

    DT.at[d,z1] = DT.at[d,z1] + 1
    WT.at[w,z1] = WT.at[w,z1] + 1
    
    return z1

### Generate model

In [ ]:
tokens = init_topics(tokens)
DT = get_DT(tokens) + alpha
WT = get_WT(tokens) + beta
for i in range(n_iters):
    print(i, end=' ')
    tokens['topic_id'] = tokens.apply(get_new_topic, 1)
print('Done')

In [140]:
print_topics(WT, 10)

TOPIC 0: drive mercury based nasa disks file possible likely according colorado
TOPIC 1: thing israel internet using remember lebanon robert try bosnian original
TOPIC 2: might soc system christopher branch ultra read put msg heavy
TOPIC 3: etc true body long save probably allergic happy buy perfect
TOPIC 4: better case please support yeast boston executive general responsible course
TOPIC 5: luke opinions least subject email government west suggest net sci
TOPIC 6: culture non graphics due compound knowledge far life windows computer
TOPIC 7: article writes world corn find left military posted gordon tanks
TOPIC 8: information already biden ago chips maybe embargo listserv area start
TOPIC 9: post around fine article bonds little willing request fast orbit
TOPIC 10: technology bad point anything agents getting word place apparently security
TOPIC 11: card may isa bit seems mode evidence account sodium matthew
TOPIC 12: writes article last christian almost david astronomy deal simple b

## METHOD 2

In [177]:
def init_phi(wt):
    n = np.ones(wt.shape[0])
    phi = wt.apply(lambda x: pd.Series(np.random.dirichlet(beta * n)))
    phi.index.name = 'term_id'
    phi.index = wt.index
    return phi

In [178]:
def print_topics(wt, n=5):    
    wtx = (WT / WT.sum())
    wtx['term_str'] = vocab.term
    wtx = wtx.set_index('term_str')
    for t in topics.index:
        T = wtx[t]
#         T = T[T > alpha]
        print('TOPIC', t,  end=': ')
        try:
            print(' '.join(T.sort_values(ascending=False).head(n).index.values))
        except:
            print("NO DICE")

### Row Function

In [ ]:
def method_2(row):
    
    d = row.name[0] # doc_id
    k = row.name[1] # token_num
    z = row.topic_id
    w = row.term_id
    
    p_z = np.exp(np.log(THETA.loc[d]) + np.log(PHI.loc[w]))
    p_z = THETA.loc[d] * PHI.loc[w]
    p_z = p_z / np.sum(p_z)
    z1 = np.random.multinomial(1, p_z).argmax()
    
    return z1

### Generate model

In [ ]:
tokens = init_topics(tokens)
DT = get_DT(tokens)
WT = get_WT(tokens)
THETA = DT.apply(lambda x: pd.Series(np.random.dirichlet(alpha + x)), 1)
PHI = WT.apply(lambda x: pd.Series(np.random.dirichlet(beta + x)))
PHI.index = WT.index

for i in range(n_iters):
    print(i, end=' ')
    tokens['topic_id'] = tokens.apply(method_2, 1)
    DT = get_DT(tokens)
    WT = get_WT(tokens)
    THETA = DT.apply(lambda x: pd.Series(np.random.dirichlet(alpha + x)), 1)
    PHI = WT.apply(lambda x: pd.Series(np.random.dirichlet(beta + x)))
    PHI.index = WT.index

print()
print_topics(WT, 10)

## METHOD 3

### Compute `P(topic|document)`

`p(topic t | document d)` Where proportion of words in document d that are assigned to topic t

In [ ]:
def get_p_tGd(tokens):
    p_tGd = tokens.reset_index().groupby(['doc_id','topic_id']).token_num.count().to_frame().unstack().fillna(0)
    p_tGd = p_tGd.apply(lambda x: x / x.sum(), axis=1)
    p_tGd.columns = p_tGd.columns.droplevel(0)
    return p_tGd

In [ ]:
p_tGd = get_p_tGd(tokens)

In [ ]:
p_tGd.head()

### Compute `P(word|topic)`
`p(word w | topic t)` Where proportion of assignments to topic t, over all documents d, that come from word w

In [ ]:
def get_p_wGt(tokens):
    p_wGt = tokens.reset_index().groupby(['topic_id','term_id']).token_num.count().to_frame().unstack().fillna(0)
    p_wGt = p_wGt.apply(lambda x: x / x.sum(), axis=1)
    p_wGt.columns = p_wGt.columns.droplevel(0)
    return p_wGt

In [ ]:
p_wGt = get_p_wGt(tokens)

In [ ]:
p_wGt

### Get best topic 

`p(topic t’ | document d) * p(word w | topic t’)`

In [ ]:
def get_best_topic(row, ptd, pwt):
    doc_id = row.name[0]
    token_num = row.name[1]
    term_id = row.term_id
    results = [ptd.at[topic_id, doc_id] * pwt.at[term_id, topic_id] for topic_id in topics.index]
    new_topic = results.index(max(results))
    return new_topic

In [ ]:
tokens = init_topics(tokens)
for i in range(n_iters):
    print(i, end=' ')
    DT = get_DT(tokens)
    WT = get_WT(tokens)
    PTD = (DT.T / DT.T.sum())
    PWT = (WT / WT.sum())
    tokens['topic_id'] = tokens.apply(lambda row: get_best_topic(row, PTD, PWT), axis=1)
print('Done')

### Show Topics

In [ ]:
TOPICS = tokens.groupby(['topic_id','term_id']).term_id.count().to_frame().unstack().fillna(0).T
TOPICS.index = TOPICS.index.droplevel(0)
TOPICS['term_str'] = vocab.term

In [ ]:
for topic_id in topics.index:
    print(topic_id, ', '.join(TOPICS.sort_values(topic_id, ascending=False).head(10).term_str.values))